<a href="https://colab.research.google.com/github/GitMishka/1/blob/main/Paylocity_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import paramiko
import os
import csv
import psycopg2
# SFTP Configuration
hostname = os.environ.get('FILES_HOST')
username = os.environ.get('FILES_USER')
password = os.environ.get('FILES_PASSWORD')
port = 22
remote_dir_path = "/yardi-qualtrics/testsftp"
local_dir_path = "./downloaded_files"

DB_PARAMS = {
    "dbname": os.environ.get('AZURE_DB_NAME'),
    "user": os.environ.get('AZURE_DB_USER'),
    "password": os.environ.get('AZURE_DB_PASSWORD'),
    "host": os.environ.get('AZURE_DB_HOST'),
    "port": "5432"
}

def insert_data_into_db(file_path):
    conn = None
    try:
        conn = psycopg2.connect(**DB_PARAMS)
        cursor = conn.cursor()

        with open(file_path, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:

                cursor.execute("""INSERT INTO public.paylocityreport ("Community", "Date", "ED", "BOD", "DON", "FSD", "LED", "LPD", "CRD", "MAINT") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", row)

        conn.commit()
        cursor.close()
        print(f"Data from {file_path} inserted into the database successfully.")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    ssh_client.connect(hostname, port, username, password)
    sftp = ssh_client.open_sftp()

    if not os.path.exists(local_dir_path):
        os.makedirs(local_dir_path)

    file_list = sftp.listdir(remote_dir_path)
    csv_files = [file for file in file_list if file.endswith('.csv')]

    for file in csv_files:
        remote_file_path = os.path.join(remote_dir_path, file)
        local_file_path = os.path.join(local_dir_path, file)

        sftp.get(remote_file_path, local_file_path)
        print(f"File {file} downloaded successfully")

        sftp.remove(remote_file_path)
        print(f"File {file} removed from remote server")

        insert_data_into_db(local_file_path)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if 'sftp' in locals():
        sftp.close()
    ssh_client.close()
